In [7]:
from datetime import datetime, timedelta
import v20
import numpy as np
import math
import time

In [8]:
def getCandles(api, fromTime, toTime):

    response = api.instrument.candles(instrument = "AUD_USD"
                                     , granularity = "M1"
                                     , fromTime = fromTime
                                     , toTime = toTime
                                     , smooth = True
                                    )

    candles = response.get("candles", 200)
    
    return np.array([(x.mid.o + x.mid.c +x.mid.h + x.mid.l) / 4 for x in candles])

In [9]:
def compareAvgs(price_mid_array, prop_pos):
    
    len_all = len(price_mid_array)
    len_pos = math.ceil(len_all * .3)
    len_pre = len_all - len_pos

    avg_pre = np.mean(price_mid_array[:len_pre])
    avg_pos = np.mean(price_mid_array[len_pre:])

    return avg_pos - avg_pre

In [10]:
# token and account_id
with open("../../../global/token_demo.txt") as T:
    token = T.read().strip()
with open("../../../global/account_demo.txt") as T:
    account = T.read().strip()

# start time
curTime = datetime.utcnow()
# stop time
stopTime = datetime.utcnow() + timedelta(hours = 2)

# candle history time frame
fromTime = datetime.utcnow() - timedelta(hours = 1) # 1 hour of history (with time interval 1 min)
fromTime = fromTime.isoformat('T') + 'Z'
toTime = datetime.utcnow().isoformat('T') + 'Z'

# post period proportion
prop_pos = .2

api = v20.Context(hostname = "api-fxpractice.oanda.com"
                 , token = token)

In [ ]:
################################ execution ################################

while curTime <= stopTime:
    print("Start Time: {}".format(curTime))
    # sleep
    time.sleep(5 * 60)
    
    # me
    me = api.account.summary(account).get("account")

    # connect api
    api = v20.Context(hostname = "api-fxpractice.oanda.com", token = token)
    
    # get candles
    price_mid_array = getCandles(api, fromTime, toTime)
    
    # price difference between pre and post periods
    price_diff = compareAvgs(price_mid_array, prop_pos)
    
    # last price
    price_last = price_mid_array[len(price_mid_array) - 1]
    
    # buy/sell units
    units = math.ceil(me.balance * .001 / price_last)
    
    # simple strategy
    if price_diff > 0:
        response = api.order.market(accountID = account
                                    , instrument = "AUD_USD"
                                    , type = "sell"
                                    , units = units)

        print("Sell Order {} at ${} for {} units".format(response.reason, price_last, units))

    elif price_diff < 0:
        response = api.order.market(accountID = account
                                    , instrument = "AUD_USD"
                                    , side = "buy"
                                    , units = units)

        print("Sell Order {} at ${} for {} units".format(response.reason, price_last, units))
    
    # update curTime
    curTime = datetime.utcnow()

Start Time: 2017-03-13 08:24:14.751212
